## Fake News and Real News Classification
**Kan Zhou and Minhui Ma**

### 1. Import Packages and Load Data

In [1]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import re

In [2]:
# load data
real_data = pd.read_csv('True.csv')
fake_data = pd.read_csv('Fake.csv')

In [3]:
real_data.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [4]:
fake_data.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
# add column "target"
# 1 for real news, 0 for fake news
real_data['target'] = 1
fake_data['target'] = 0 

# merge the 2 datasets
data = pd.concat([real_data, fake_data], ignore_index=True, sort=False)
data.head()

,title,text,subject,date,target
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


### 2. Data Cleaning

#### 2.1. Data Information

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   target   44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.7+ MB


In [7]:
print("The number of real news is %d" %len(real_data))
print("The number of fake news is %d" %len(fake_data))

The number of real news is 21417
The number of fake news is 23481


The dataset has 5 features and 44898 observations with no missing values. 4 of the features are object and 1 feature is integer. 21417 of the observations are real news and 23481 of the observations are fake news.

#### 2.2. Remove Non-letters and Change to Lowercase

In [8]:
# remove non-letters and change to lower case
def clean_text(string_in):
    string_in = re.sub(r"[^A-Za-z0-9\-]", " ", string_in).lower().split()  # Replace all non-letters with spaces
    string_in = string_in.lower()                         # Tranform to lower case        
    return string_in.strip()

data['title'] = data.title.apply(clean_text)
data['text'] = data.text.apply(clean_text)
data.head()

,title,text,subject,date,target
0,as u s budget fight looms republicans flip t...,washington reuters - the head of a conservat...,politicsNews,"December 31, 2017",1
1,u s military to accept transgender recruits o...,washington reuters - transgender people will...,politicsNews,"December 29, 2017",1
2,senior u s republican senator let mr muell...,washington reuters - the special counsel inv...,politicsNews,"December 31, 2017",1
3,fbi russia probe helped by australian diplomat...,washington reuters - trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,trump wants postal service to charge much mor...,seattle washington reuters - president donal...,politicsNews,"December 29, 2017",1


In [9]:
# "U.S." was changed to "u s" in the previous step
# change "u s" to U.S.
data['title'] = data['title'].str.replace("u s", "u.s.")
data['text'] = data['text'].str.replace("u s", "u.s.")

# remove the date column
del data['date']
data.head()

,title,text,subject,target
0,as u.s. budget fight looms republicans flip ...,washington reuters - the head of a conservat...,politicsNews,1
1,u.s. military to accept transgender recruits ...,washington reuters - transgender people will...,politicsNews,1
2,senior u.s. republican senator let mr muel...,washington reuters - the special counsel inv...,politicsNews,1
3,fbi russia probe helped by australian diplomat...,washington reuters - trump campaign adviser ...,politicsNews,1
4,trump wants postal service to charge much mor...,seattle washington reuters - president donal...,politicsNews,1


#### 2.3. Remove Stopwords

In [10]:
# nltk.download("stopwords")

In [11]:
from nltk.corpus import stopwords  
stopWords = set(stopwords.words('english'))

def stopwords(string_in):
    string_in = nltk.word_tokenize(str(string_in))
    string_in = [word for word in string_in if not word in stopWords]
    return string_in
data['title'] = data.title.apply(stopwords)
data['text'] = data.text.apply(stopwords)
data.head()

,title,text,subject,target
0,"[u.s., budget, fight, looms, republicans, flip...","[washington, reuters, -, head, conservative, r...",politicsNews,1
1,"[u.s., military, accept, transgender, recruits...","[washington, reuters, -, transgender, people, ...",politicsNews,1
2,"[senior, u.s., republican, senator, let, mr, m...","[washington, reuters, -, special, counsel, inv...",politicsNews,1
3,"[fbi, russia, probe, helped, australian, diplo...","[washington, reuters, -, trump, campaign, advi...",politicsNews,1
4,"[trump, wants, postal, service, charge, much, ...","[seattle, washington, reuters, -, president, d...",politicsNews,1


#### 2.4. Lemmatization

In [12]:
# nltk.download('wordnet')

In [15]:
lemma = nltk.WordNetLemmatizer()

def lemmatization(string_in):
    string_in = [lemma.lemmatize(word) for word in string_in if not word=='-']
    return string_in
data['title'] = data.title.apply(lemmatization)
data['text'] = data.text.apply(lemmatization)
data.head()

,title,text,subject,target
0,"[u.s., budget, fight, loom, republican, flip, ...","[washington, reuters, head, conservative, repu...",politicsNews,1
1,"[u.s., military, accept, transgender, recruit,...","[washington, reuters, transgender, people, all...",politicsNews,1
2,"[senior, u.s., republican, senator, let, mr, m...","[washington, reuters, special, counsel, invest...",politicsNews,1
3,"[fbi, russia, probe, helped, australian, diplo...","[washington, reuters, trump, campaign, adviser...",politicsNews,1
4,"[trump, want, postal, service, charge, much, a...","[seattle, washington, reuters, president, dona...",politicsNews,1


In [16]:
data.text[0]

['washington',
 'reuters',
 'head',
 'conservative',
 'republican',
 'faction',
 'u.s.',
 'congress',
 'voted',
 'month',
 'huge',
 'expansion',
 'national',
 'debt',
 'pay',
 'tax',
 'cut',
 'called',
 'fiscal',
 'conservative',
 'sunday',
 'urged',
 'budget',
 'restraint',
 'keeping',
 'sharp',
 'pivot',
 'way',
 'among',
 'republican',
 'u.s.',
 'representative',
 'mark',
 'meadow',
 'speaking',
 'cbs',
 'face',
 'nation',
 'drew',
 'hard',
 'line',
 'federal',
 'spending',
 'lawmaker',
 'bracing',
 'battle',
 'january',
 'return',
 'holiday',
 'wednesday',
 'lawmaker',
 'begin',
 'trying',
 'pa',
 'federal',
 'budget',
 'fight',
 'likely',
 'linked',
 'issue',
 'immigration',
 'policy',
 'even',
 'november',
 'congressional',
 'election',
 'campaign',
 'approach',
 'republican',
 'seek',
 'keep',
 'control',
 'congress',
 'president',
 'donald',
 'trump',
 'republican',
 'want',
 'big',
 'budget',
 'increase',
 'military',
 'spending',
 'democrat',
 'also',
 'want',
 'proportional'

### 3. Data Visualization

#### 3.1. Data Information

#### 3.2. Word Cloud

### 4. Classification